# --------------------------------------------------------------------------------------------------------------
# PLAYAS de ESPAÑA 2017 : Neo4j
# --------------------------------------------------------------------------------------------------------------

## Inicializaciones previas

### a) Carga de librerías

In [1]:
import pandas as pd
import datetime
import json

# The py2neo.database package contains classes and functions required to interact with a Neo4j server. 
# The most important of these is the Graph class which represents a Neo4j graph database instance and 
# provides access to a large portion of the most commonly used py2neo API.

from py2neo import Graph, Node, Relationship 

# A) Carga de datos desde Excel a un DataFrame de PANDAS

Leemos los datos desde el fichero Excel y los desnormalizamos : leemos las 2 hojas del fichero Excel en un **mismo** DataFrame de Pandas para guardarlo todo junto.

In [2]:
df_excel_provincias = pd.read_excel("./data/playas_2017.xlsx", sheetname= "HOJA_PROVINCIAS")
df_excel_playas     = pd.read_excel("./data/playas_2017.xlsx", sheetname= "HOJA_PLAYAS")

In [3]:
df_neo4j = pd.merge(df_excel_provincias, df_excel_playas, on = ['id_provincia'], how = 'inner')
df_neo4j.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3511 entries, 0 to 3510
Data columns (total 12 columns):
id_provincia       3511 non-null int64
nom_provincia      3511 non-null object
comunidad          3511 non-null object
cod_playa          3511 non-null int64
nom_playa          3511 non-null object
localidad          3511 non-null object
clave_nom_playa    3511 non-null object
longitud           3511 non-null int64
longitud_txt       3511 non-null object
arena              3511 non-null object
nudista            3511 non-null object
bandera_azul       3511 non-null object
dtypes: int64(3), object(9)
memory usage: 356.6+ KB


## B.1) Conexión a la BD de Neo4j

# B) BD de Neo4j

### B.1.1) Carga de CYPHER

In [4]:
%load_ext cypher

### B.1.2) Conexión con Neo4j

In [5]:
graph_N = Graph()

### B.1.3) Borrado de datos

In [6]:
# Borrado de todos los nodos y relaciones
graph_N.delete_all()

# C) Carga de datos en Neo4j

## C.1) Transformaciones previas

La carga se realizará en Neo a partir de la lista de datos en formato JSON, generados con Pandas

In [7]:
json_string = df_neo4j.to_json(orient = 'records')
json_list = json.loads(json_string)

En Neo se van a crear **3 tipos de nodos** y **2 tipos de relaciones** :

<br><br> 

<img src="images/Modelo_BD___Neo4j.JPG",width=600,height=500>

<br><br> 

### Definimos los siguientes **NODOS** y sus **atributos** :

**1) Nodo Playa**
   + Nombre de la playa
   + Código de la playa
   + localidad de la playa
   + Identificación completa de la playa : playa (localidad)
   + longitud (en número) de la playa
   + longitud (en texto)  de la playa
   + Tipo de arena de la playa
   + Si es una playa nudista
   + Si es una playa con bandera azul   
   
   
**2) Nodo Provincia**
   + Nombre de la provincia


**3) Nodo Comunidad Autónoma**
   + Nombre de la Comunidad Autónoma



### Definimos las siguientes **RELACIONES** (sin **atributos**) :

**1) Relación UBICADA** : en qué **provincia** está ubicada una determinada **playa**.

**2) Relación PERTENECE** : a qué **comunidad autónoma** pertenece una determinada **provincia**.

## C.2) Inserción de los datos en Neo4j

Creamos los **nodos** y las **relaciones** en Neo4j simplemente iterando sobre la lista JSON.

In [8]:

for index, playa_json in enumerate(json_list):
    
    #-----------------------------------------------------------------------
    # NODOS : definición
    #-----------------------------------------------------------------------
    var_playa     = Node("nodo_playa", 
                         np_nombre          = playa_json["nom_playa"],
                         np_cod_playa       = playa_json["cod_playa"],
                         np_localidad       = playa_json["localidad"],
                         np_clave_nom_playa = playa_json["clave_nom_playa"],
                         np_longitud        = playa_json["longitud"],
                         np_longitud_txt    = playa_json["longitud_txt"],
                         np_arena           = playa_json["arena"],
                         np_nudista         = playa_json["nudista"],
                         np_bandera_azul    = playa_json["bandera_azul"],
                        )
    var_provincia = Node("nodo_provincia", 
                         nv_nombre = playa_json["nom_provincia"]
                        )
    var_comunidad = Node("nodo_comunidad", 
                         nc_nombre = playa_json["comunidad"]
                        )
    
    #-----------------------------------------------------------------------
    # RELACIONES : definición
    #-----------------------------------------------------------------------         
    rel_playa_provincia     = Relationship(var_playa,      "UBICADA",   var_provincia)
    rel_provincia_comunidad = Relationship(var_provincia,  "PERTENECE", var_comunidad)

    
    #-----------------------------------------------------------------------
    # Creación de NODOS y RELACIONES
    #-----------------------------------------------------------------------       
    graph_N.merge(var_playa | var_provincia | var_comunidad | rel_playa_provincia | rel_provincia_comunidad )   
    
    if index % 500 == 0:
        print index
        

0
500
1000
1500
2000
2500
3000
3500


### Para evitar problemas de acentos y caracteres raros ...

**NOTA** : lo pongo aquí, porque si lo pongo antes del bucle "for" anterior (el de carga de datos) no va imprimiendo el valor de "index" (no sé por qué razón).

In [9]:
# Para eliminar probemas de acentos y demás caracteres raros

import sys
reload (sys)

sys.setdefaultencoding('utf-8')

### Comprobaciones previas ...

Número de playas ...

In [10]:
%%cypher

MATCH (:nodo_playa)
RETURN count(*)

count(*)
3511


Número de provincias ...

In [11]:
%%cypher

MATCH (:nodo_provincia)
RETURN count(*)

count(*)
24


Número de comunidades ...

In [12]:
%%cypher

MATCH (:nodo_comunidad)
RETURN count(*)

count(*)
12


All good!

# ------------------------------------------------------------------------------
# QUERYS   (Neo4j)
# ------------------------------------------------------------------------------

Todas las **QUERYS** se pueden ejecutar de **3 formas** diferentes :

 1. Con **CYPHER** directamente.
 2. Con **graph.run()** convirtiendo su resultado **"py2neo.database.Cursor"** en un **DataFrame** de PANDAS (mediante una función definida ad hoc).
 3. Con **CYPHER** convirtiendo su resultado **"cypher.run.ResultSet"** en un **DataFrame** de PANDAS (mediante el método **"get_dataframe()"**).



Vamos a necesitar las siguientes **FUNCIONES** :

#### 1) Función para convertir RESULTADOS (necesaria para la ejecución de tipo 2) :

   + Utilizaremos la función **graph.run()** que nod devuelve un cursor a los datos que nos devuelve la BD.

   + La siguiente función **convierte** ese **resultado devuelto por la base de datos** en un **DataFrame** de Pandas.

In [13]:
def fun_convertir_cursorBD_a_dataframe(data):
    df = pd.DataFrame(list(data), columns = data.keys())
    return df

#### 2) Función para convertir FECHAS (necesaria para las ejecuciones de tipo 2 y 3, las de DataFrames)  : 

+ Neo4j, en su versión actual, no puede guardar datos de tipo fecha en los atributos, por lo que se almacena en formato TIMESTAMP de Unix (el formato original).  

+ La siguiente función **convierte** una columna de un **DataFrame** en formato **TIMESTAMP a Fecha**.

In [14]:
def transform_date(df, column_name):
    if column_name in df.columns:
        # Las fechas están en formato UNIX TIMESTAMP. Las volvemos a convertir a formato Date...
        df[column_name] = pd.to_datetime(df[column_name], unit = 'ms')
        
    return df

### QUERY_1) Las 10 playas más largas (incluída la provincia)¶

#### 1.1) Con CYPHER

In [15]:
%%cypher

MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad)
RETURN 
       provincia.nv_nombre       AS PROVINCIA,
       playa.np_nombre           AS PLAYA, 
       playa.np_localidad        AS LOCALIDAD,
       playa.np_clave_nom_playa  AS CLAVE_NOM_PLAYA,
       playa.np_longitud         AS LONGITUD,
       playa.np_longitud_txt     AS LONGITUD_TXT,
       playa.np_arena            AS ARENA,
       playa.np_nudista          AS NUDISTA,
       playa.np_bandera_azul     AS BANDERA_AZUL
ORDER BY playa.np_longitud DESC
LIMIT 10

PROVINCIA,PLAYA,LOCALIDAD,CLAVE_NOM_PLAYA,LONGITUD,LONGITUD_TXT,ARENA,NUDISTA,BANDERA_AZUL
Huelva,Doñana,Almonte,Almonte (Doñana),28000,28.000 metros,Dorada,No,No
Huelva,Castilla,Almonte,Almonte (Castilla),17000,17.000 metros,Dorada,No,No
Las Palmas,El Cofete,Pájara,Pájara (El Cofete),13700,13.700 metros,Dorada,Sí,No
Huelva,Nueva Umbría,Lepe,Lepe (Nueva Umbría),12000,12.000 metros,Dorada,Sí,No
Huelva,Mazagón,Palos de la Frontera,Palos de la Frontera (Mazagón),9000,9.000 metros,Dorada,No,No
Almería,Punta Entinas Sabinar,El Ejido,El Ejido (Punta Entinas Sabinar),6500,6.500 metros,Oscura,No,No
Tarragona,Punta del Fangar,Deltebre,Deltebre (Punta del Fangar),6500,6.500 metros,Dorada,Sí,No
Girona,Sant Pere Pescador,Sant Pere Pescador,Sant Pere Pescador (Sant Pere Pescador),6400,6400 metros,Dorada,Sí,No
Tarragona,El Trabucador,Sant Carles de la Ràpita,Sant Carles de la Ràpita (El Trabucador),6165,6.165 metros,Dorada,No,No
A Coruña,Carnota,Carnota,Carnota (Carnota),5600,5.600 metros,Blanca,No,No


#### 1.2) Con "graph.run()" convirtiendo su resultado en un DataFrame

In [16]:
query = """
MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad)
RETURN 
       provincia.nv_nombre       AS PROVINCIA,
       playa.np_nombre           AS PLAYA, 
       playa.np_localidad        AS LOCALIDAD,
       playa.np_clave_nom_playa  AS CLAVE_NOM_PLAYA,
       playa.np_longitud         AS LONGITUD,
       playa.np_longitud_txt     AS LONGITUD_TXT,
       playa.np_arena            AS ARENA,
       playa.np_nudista          AS NUDISTA,
       playa.np_bandera_azul     AS BANDERA_AZUL
ORDER BY playa.np_longitud DESC
LIMIT 10
""" 

data = graph_N.run(query)
type(data)

py2neo.database.Cursor

In [17]:
df = fun_convertir_cursorBD_a_dataframe(data)
df

,PROVINCIA,PLAYA,LOCALIDAD,CLAVE_NOM_PLAYA,LONGITUD,LONGITUD_TXT,ARENA,NUDISTA,BANDERA_AZUL
0,Huelva,Doñana,Almonte,Almonte (Doñana),28000,28.000 metros,Dorada,No,No
1,Huelva,Castilla,Almonte,Almonte (Castilla),17000,17.000 metros,Dorada,No,No
2,Las Palmas,El Cofete,Pájara,Pájara (El Cofete),13700,13.700 metros,Dorada,Sí,No
3,Huelva,Nueva Umbría,Lepe,Lepe (Nueva Umbría),12000,12.000 metros,Dorada,Sí,No
4,Huelva,Mazagón,Palos de la Frontera,Palos de la Frontera (Mazagón),9000,9.000 metros,Dorada,No,No
5,Almería,Punta Entinas Sabinar,El Ejido,El Ejido (Punta Entinas Sabinar),6500,6.500 metros,Oscura,No,No
6,Tarragona,Punta del Fangar,Deltebre,Deltebre (Punta del Fangar),6500,6.500 metros,Dorada,Sí,No
7,Girona,Sant Pere Pescador,Sant Pere Pescador,Sant Pere Pescador (Sant Pere Pescador),6400,6400 metros,Dorada,Sí,No
8,Tarragona,El Trabucador,Sant Carles de la Ràpita,Sant Carles de la Ràpita (El Trabucador),6165,6.165 metros,Dorada,No,No
9,A Coruña,Carnota,Carnota,Carnota (Carnota),5600,5.600 metros,Blanca,No,No


#### 1.3) Con **CYPHER** convirtiendo su resultado en un DataFrame

In [18]:
res = %cypher MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad) \
              RETURN                                                  \
                     provincia.nv_nombre       AS PROVINCIA,          \
                     playa.np_nombre           AS PLAYA,              \
                     playa.np_localidad        AS LOCALIDAD,          \
                     playa.np_clave_nom_playa  AS CLAVE_NOM_PLAYA,    \
                     playa.np_longitud         AS LONGITUD,           \
                     playa.np_longitud_txt     AS LONGITUD_TXT,       \
                     playa.np_arena            AS ARENA,              \
                     playa.np_nudista          AS NUDISTA,            \
                     playa.np_bandera_azul     AS BANDERA_AZUL        \
              ORDER BY playa.np_longitud DESC                         \
              LIMIT 10

            
type(res)

cypher.run.ResultSet

In [19]:
res.get_dataframe()

,PROVINCIA,PLAYA,LOCALIDAD,CLAVE_NOM_PLAYA,LONGITUD,LONGITUD_TXT,ARENA,NUDISTA,BANDERA_AZUL
0,Huelva,Doñana,Almonte,Almonte (Doñana),28000,28.000 metros,Dorada,No,No
1,Huelva,Castilla,Almonte,Almonte (Castilla),17000,17.000 metros,Dorada,No,No
2,Las Palmas,El Cofete,Pájara,Pájara (El Cofete),13700,13.700 metros,Dorada,Sí,No
3,Huelva,Nueva Umbría,Lepe,Lepe (Nueva Umbría),12000,12.000 metros,Dorada,Sí,No
4,Huelva,Mazagón,Palos de la Frontera,Palos de la Frontera (Mazagón),9000,9.000 metros,Dorada,No,No
5,Almería,Punta Entinas Sabinar,El Ejido,El Ejido (Punta Entinas Sabinar),6500,6.500 metros,Oscura,No,No
6,Tarragona,Punta del Fangar,Deltebre,Deltebre (Punta del Fangar),6500,6.500 metros,Dorada,Sí,No
7,Girona,Sant Pere Pescador,Sant Pere Pescador,Sant Pere Pescador (Sant Pere Pescador),6400,6400 metros,Dorada,Sí,No
8,Tarragona,El Trabucador,Sant Carles de la Ràpita,Sant Carles de la Ràpita (El Trabucador),6165,6.165 metros,Dorada,No,No
9,A Coruña,Carnota,Carnota,Carnota (Carnota),5600,5.600 metros,Blanca,No,No


### QUERY_2) Las 10 playas más largas de una cierta provincia (incluído el nombre de la provincia)

#### 2.1) Con CYPHER

In [20]:
%%cypher

MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad)
WHERE provincia.nv_nombre = "A Coruña"
RETURN 
       playa.np_nombre         AS PLAYA, 
       playa.np_longitud       AS LONGITUD,
       provincia.nv_nombre     AS PROVINCIA,
       comunidad.nc_nombre     AS COMUNIDAD
ORDER BY playa.np_longitud DESC
LIMIT 10

PLAYA,LONGITUD,PROVINCIA,COMUNIDAD
Carnota,5600,A Coruña,Galicia
Baldaio,4000,A Coruña,Galicia
A Frouxeira,3000,A Coruña,Galicia
Vilar,2900,A Coruña,Galicia
Morouzos,2800,A Coruña,Galicia
Traba,2650,A Coruña,Galicia
Barraña,2150,A Coruña,Galicia
Areas Longas,2100,A Coruña,Galicia
Carragueiros,2000,A Coruña,Galicia
Langosteira,1970,A Coruña,Galicia


#### 2.2) Con "graph.run()" convirtiendo su resultado en un DataFrame

In [21]:
query = """
MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad)
WHERE provincia.nv_nombre = 'A Coruña'
RETURN 
       playa.np_nombre         AS PLAYA, 
       playa.np_longitud       AS LONGITUD,
       provincia.nv_nombre     AS PROVINCIA,
       comunidad.nc_nombre     AS COMUNIDAD
ORDER BY playa.np_longitud DESC
LIMIT 10
""" 

data = graph_N.run(query)
df = fun_convertir_cursorBD_a_dataframe(data)
df

,PLAYA,LONGITUD,PROVINCIA,COMUNIDAD
0,Carnota,5600,A Coruña,Galicia
1,Baldaio,4000,A Coruña,Galicia
2,A Frouxeira,3000,A Coruña,Galicia
3,Vilar,2900,A Coruña,Galicia
4,Morouzos,2800,A Coruña,Galicia
5,Traba,2650,A Coruña,Galicia
6,Barraña,2150,A Coruña,Galicia
7,Areas Longas,2100,A Coruña,Galicia
8,Carragueiros,2000,A Coruña,Galicia
9,Langosteira,1970,A Coruña,Galicia


#### 2.3) Con **CYPHER** convirtiendo su resultado en un DataFrame

In [22]:
res = %cypher MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad) \
              WHERE provincia.nv_nombre = 'A Coruña'            \
              RETURN                                            \
                     playa.np_nombre         AS PLAYA,          \
                     playa.np_longitud       AS LONGITUD,       \
                     provincia.nv_nombre     AS PROVINCIA,      \
                     comunidad.nc_nombre     AS COMUNIDAD       \
              ORDER BY playa.np_longitud DESC                   \
              LIMIT 10

            
res.get_dataframe()

,PLAYA,LONGITUD,PROVINCIA,COMUNIDAD
0,Carnota,5600,A Coruña,Galicia
1,Baldaio,4000,A Coruña,Galicia
2,A Frouxeira,3000,A Coruña,Galicia
3,Vilar,2900,A Coruña,Galicia
4,Morouzos,2800,A Coruña,Galicia
5,Traba,2650,A Coruña,Galicia
6,Barraña,2150,A Coruña,Galicia
7,Areas Longas,2100,A Coruña,Galicia
8,Carragueiros,2000,A Coruña,Galicia
9,Langosteira,1970,A Coruña,Galicia


### QUERY_3) Las playas de una comunidad autónoma concreta (ordenadas por longitud)

#### 3.1) Con CYPHER

In [23]:
%%cypher

MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad)
WHERE comunidad.nc_nombre = 'Galicia'
RETURN 
       playa.np_nombre         AS PLAYA, 
       playa.np_longitud       AS LONGITUD,
       provincia.nv_nombre     AS PROVINCIA,
       comunidad.nc_nombre     AS COMUNIDAD
ORDER BY playa.np_longitud DESC
LIMIT 10

PLAYA,LONGITUD,PROVINCIA,COMUNIDAD
Carnota,5600,A Coruña,Galicia
Baldaio,4000,A Coruña,Galicia
A Frouxeira,3000,A Coruña,Galicia
Vilar,2900,A Coruña,Galicia
Morouzos,2800,A Coruña,Galicia
Traba,2650,A Coruña,Galicia
Da Punta,2400,Pontevedra,Galicia
A Lanzada,2400,Pontevedra,Galicia
Barraña,2150,A Coruña,Galicia
Areas Longas,2100,A Coruña,Galicia


#### 3.2) Con "graph.run()" convirtiendo su resultado en un DataFrame

In [24]:
query = """
MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad)
WHERE comunidad.nc_nombre = 'Galicia'
RETURN 
       playa.np_nombre         AS PLAYA, 
       playa.np_longitud       AS LONGITUD,
       provincia.nv_nombre     AS PROVINCIA,
       comunidad.nc_nombre     AS COMUNIDAD
ORDER BY playa.np_longitud DESC
LIMIT 10
""" 

data = graph_N.run(query)
df = fun_convertir_cursorBD_a_dataframe(data)
df

,PLAYA,LONGITUD,PROVINCIA,COMUNIDAD
0,Carnota,5600,A Coruña,Galicia
1,Baldaio,4000,A Coruña,Galicia
2,A Frouxeira,3000,A Coruña,Galicia
3,Vilar,2900,A Coruña,Galicia
4,Morouzos,2800,A Coruña,Galicia
5,Traba,2650,A Coruña,Galicia
6,Da Punta,2400,Pontevedra,Galicia
7,A Lanzada,2400,Pontevedra,Galicia
8,Barraña,2150,A Coruña,Galicia
9,Areas Longas,2100,A Coruña,Galicia


#### 3.3) Con **CYPHER** convirtiendo su resultado en un DataFrame

In [25]:
res = %cypher MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad) \
              WHERE comunidad.nc_nombre = 'Galicia'             \
              RETURN                                            \
                     playa.np_nombre         AS PLAYA,          \
                     playa.np_longitud       AS LONGITUD,       \
                     provincia.nv_nombre     AS PROVINCIA,      \
                     comunidad.nc_nombre     AS COMUNIDAD       \
              ORDER BY playa.np_longitud DESC                   \
              LIMIT 10
            
res.get_dataframe()

,PLAYA,LONGITUD,PROVINCIA,COMUNIDAD
0,Carnota,5600,A Coruña,Galicia
1,Baldaio,4000,A Coruña,Galicia
2,A Frouxeira,3000,A Coruña,Galicia
3,Vilar,2900,A Coruña,Galicia
4,Morouzos,2800,A Coruña,Galicia
5,Traba,2650,A Coruña,Galicia
6,Da Punta,2400,Pontevedra,Galicia
7,A Lanzada,2400,Pontevedra,Galicia
8,Barraña,2150,A Coruña,Galicia
9,Areas Longas,2100,A Coruña,Galicia


### QUERY_4) Las 10 provincias que mas metros de playa tienen¶

#### 4.1) Con CYPHER

In [26]:
%%cypher


MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad)
RETURN 
       provincia.nv_nombre     AS PROVINCIA,
       sum(playa.np_longitud)  AS metros_de_playa
ORDER BY metros_de_playa DESC
LIMIT 10

PROVINCIA,metros_de_playa
A Coruña,157040
Cádiz,138473
Las Palmas,137337
Málaga,136720
Almería,120765
Alicante/Alacant,118148
Illes Balears,117767
Huelva,113720
Murcia,102431
Tarragona,99015


#### 4.2) Con "graph.run()" convirtiendo su resultado en un DataFrame

In [27]:
query = """
MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad)
RETURN 
       provincia.nv_nombre     AS PROVINCIA,
       sum(playa.np_longitud)  AS metros_de_playa
ORDER BY metros_de_playa DESC
LIMIT 10
""" 

data = graph_N.run(query)
df = fun_convertir_cursorBD_a_dataframe(data)
df

,PROVINCIA,metros_de_playa
0,A Coruña,157040
1,Cádiz,138473
2,Las Palmas,137337
3,Málaga,136720
4,Almería,120765
5,Alicante/Alacant,118148
6,Illes Balears,117767
7,Huelva,113720
8,Murcia,102431
9,Tarragona,99015


#### 4.3) Con **CYPHER** convirtiendo su resultado en un DataFrame

In [28]:
res = %cypher MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad) \
              RETURN                                            \
                     provincia.nv_nombre     AS PROVINCIA,      \
                     sum(playa.np_longitud)  AS metros_de_playa \
              ORDER BY metros_de_playa DESC                     \
              LIMIT 10
            
res.get_dataframe()

,PROVINCIA,metros_de_playa
0,A Coruña,157040
1,Cádiz,138473
2,Las Palmas,137337
3,Málaga,136720
4,Almería,120765
5,Alicante/Alacant,118148
6,Illes Balears,117767
7,Huelva,113720
8,Murcia,102431
9,Tarragona,99015


### QUERY_5) Metros de playa de una cierta Comunidad Autónoma agrupados por provincia¶

#### 5.1) Con CYPHER

In [29]:
%%cypher

MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad)
WHERE comunidad.nc_nombre = "Galicia"
RETURN 
       comunidad.nc_nombre     AS COMUNIDAD,
       provincia.nv_nombre     AS PROVINCIA,
       sum(playa.np_longitud)  AS metros_de_playa
ORDER BY metros_de_playa DESC
LIMIT 10

COMUNIDAD,PROVINCIA,metros_de_playa
Galicia,A Coruña,157040
Galicia,Pontevedra,93552
Galicia,Lugo,27217


#### 5.2) Con "graph.run()" convirtiendo su resultado en un DataFrame

In [30]:
query = """
MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad)
WHERE comunidad.nc_nombre = "Galicia"
RETURN 
       comunidad.nc_nombre     AS COMUNIDAD,
       provincia.nv_nombre     AS PROVINCIA,
       sum(playa.np_longitud)  AS metros_de_playa
ORDER BY metros_de_playa DESC
LIMIT 10
""" 

data = graph_N.run(query)
df = fun_convertir_cursorBD_a_dataframe(data)
df

,COMUNIDAD,PROVINCIA,metros_de_playa
0,Galicia,A Coruña,157040
1,Galicia,Pontevedra,93552
2,Galicia,Lugo,27217


#### 5.3) Con **CYPHER** convirtiendo su resultado en un DataFrame

In [31]:
res = %cypher MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad) \
              WHERE comunidad.nc_nombre = "Galicia"             \
              RETURN                                            \
                     comunidad.nc_nombre     AS COMUNIDAD,      \
                     provincia.nv_nombre     AS PROVINCIA,      \
                     sum(playa.np_longitud)  AS metros_de_playa \
              ORDER BY metros_de_playa DESC                     \
              LIMIT 10
            
res.get_dataframe()

,COMUNIDAD,PROVINCIA,metros_de_playa
0,Galicia,A Coruña,157040
1,Galicia,Pontevedra,93552
2,Galicia,Lugo,27217


### QUERY_6) ¿ Qué Comunidad tiene más metros de playa ?

#### 6.1) Con CYPHER

In [32]:
%%cypher

MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad)
RETURN 
       comunidad.nc_nombre     AS COMUNIDAD,
       sum(playa.np_longitud)  AS metros_de_playa
ORDER BY metros_de_playa DESC

COMUNIDAD,metros_de_playa
Andalucía,545459
Galicia,277809
Comunitat Valenciana,276872
Cataluña/Catalunya,237274
Canarias,209374
Illes Balears,117767
"Murcia, Región de",102431
"Asturias, Principado de",70428
Cantabria,49880
País Vasco/Euskadi,30715


#### 6.2) Con "graph.run()" convirtiendo su resultado en un DataFrame

In [33]:
query = """
MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad)
RETURN 
       comunidad.nc_nombre     AS COMUNIDAD,
       sum(playa.np_longitud)  AS metros_de_playa
ORDER BY metros_de_playa DESC
""" 

data = graph_N.run(query)
df = fun_convertir_cursorBD_a_dataframe(data)
df

,COMUNIDAD,metros_de_playa
0,Andalucía,545459
1,Galicia,277809
2,Comunitat Valenciana,276872
3,Cataluña/Catalunya,237274
4,Canarias,209374
5,Illes Balears,117767
6,"Murcia, Región de",102431
7,"Asturias, Principado de",70428
8,Cantabria,49880
9,País Vasco/Euskadi,30715


#### 6.3) Con **CYPHER** convirtiendo su resultado en un DataFrame

In [34]:
res = %cypher MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad) \
              RETURN                                            \
                     comunidad.nc_nombre     AS COMUNIDAD,      \
                     sum(playa.np_longitud)  AS metros_de_playa \
              ORDER BY metros_de_playa DESC
            
res.get_dataframe()

,COMUNIDAD,metros_de_playa
0,Andalucía,545459
1,Galicia,277809
2,Comunitat Valenciana,276872
3,Cataluña/Catalunya,237274
4,Canarias,209374
5,Illes Balears,117767
6,"Murcia, Región de",102431
7,"Asturias, Principado de",70428
8,Cantabria,49880
9,País Vasco/Euskadi,30715


### QUERY_7) ¿ Qué provincia tiene más playas ... ?

#### QUERY_7.1) ¿ Qué provincia tiene más playas ... en número ?

In [35]:
query = """
MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad)
RETURN 
       provincia.nv_nombre        AS PLAYA,
       count(playa.np_cod_playa)  AS num_playas
ORDER BY num_playas DESC
LIMIT 10
""" 

data = graph_N.run(query)
df = fun_convertir_cursorBD_a_dataframe(data)
df

,PLAYA,num_playas
0,A Coruña,410
1,Pontevedra,370
2,Illes Balears,346
3,Las Palmas,322
4,Santa Cruz de Tenerife,257
5,Asturias,205
6,Murcia,199
7,Girona,183
8,Alicante/Alacant,173
9,Málaga,131


#### QUERY_7.2) ¿ Qué provincia tiene más playas ... con nombres diferentes ?

In [36]:
query = """
MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad)
RETURN 
       provincia.nv_nombre              AS PLAYA,
       count(distinct(playa.np_nombre)) AS nombres_de_playas
ORDER BY nombres_de_playas DESC
LIMIT 10
""" 
 
data = graph_N.run(query)
df = fun_convertir_cursorBD_a_dataframe(data)
df

,PLAYA,nombres_de_playas
0,A Coruña,388
1,Pontevedra,350
2,Illes Balears,330
3,Las Palmas,303
4,Santa Cruz de Tenerife,228
5,Asturias,199
6,Murcia,193
7,Girona,179
8,Alicante/Alacant,165
9,Málaga,126


#### QUERY_7.3)  ¿ Qué provincia tiene más playas ... nudistas ?

In [37]:
query = """
MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad)
WHERE playa.np_nudista = 'Sí'
RETURN 
       provincia.nv_nombre        AS PLAYA,
       count(playa.np_cod_playa)  AS num_playas_nudistas_puras
ORDER BY num_playas_nudistas_puras DESC
LIMIT 10
""" 

data = graph_N.run(query)
df = fun_convertir_cursorBD_a_dataframe(data)
df

,PLAYA,num_playas_nudistas_puras
0,Las Palmas,146
1,Illes Balears,79
2,Santa Cruz de Tenerife,27
3,Alicante/Alacant,18
4,A Coruña,17
5,Girona,16
6,Gipuzkoa,16
7,Pontevedra,15
8,Tarragona,9
9,Barcelona,8


#### QUERY_7.4)  ¿ Qué provincia tiene más playas ... con bandera azul ?

In [38]:
query = """
MATCH (playa:nodo_playa)-[:UBICADA]-(provincia:nodo_provincia)-[:PERTENECE]-(comunidad:nodo_comunidad)
WHERE playa.np_bandera_azul = 'Sí'
RETURN 
       provincia.nv_nombre        AS PLAYA,
       count(playa.np_cod_playa)  AS num_playas_bandera_azul
ORDER BY num_playas_bandera_azul DESC
LIMIT 10
""" 

data = graph_N.run(query)
df = fun_convertir_cursorBD_a_dataframe(data)
df

,PLAYA,num_playas_bandera_azul
0,Alicante/Alacant,62
1,Pontevedra,57
2,A Coruña,45
3,Tarragona,43
4,Illes Balears,41
5,Murcia,40
6,Castellón/Castelló,33
7,Girona,31
8,Valencia/València,30
9,Barcelona,29


# FIN